# Instalando as bibliotecas


In [ ]:
!pip install openai==0.28

In [ ]:
!pip install transformers  beautifulsoup4 requests


# Declarando as bibliotecas

In [1]:
from transformers import pipeline, BertTokenizerFast, EncoderDecoderModel
import torch
import openai
import requests
from bs4 import BeautifulSoup

# Raspando dados da internet

In [2]:
def extraindo_texto_da_web(url):
    response = requests.get(url)
    response.raise_for_status()  # Verifica se o request foi bem-sucedido
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extrai todo o texto visível da página
    paragraphs = soup.find_all('p')
    text = " ".join([para.get_text() for para in paragraphs])
    return text

url = "https://pt.wikipedia.org/wiki/Microsoft_Office"

documento = extraindo_texto_da_web(url)


# Usando o BERT

## Q & R

In [3]:

qa = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")

pergunta = input("Pergunta: ")
result = qa(question=pergunta,
            context=documento,
            max_lengh=100)

print(f"Resposta: {result['answer']}\n")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu


Pergunta: Qual é a última versão Microsoft Office?
Resposta: [3]
 A versão atual do Office desktop é o Office 2019



## Summarize

In [4]:


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizerFast.from_pretrained('mrm8488/bert-mini2bert-mini-finetuned-cnn_daily_mail-summarization')
model = EncoderDecoderModel.from_pretrained('mrm8488/bert-mini2bert-mini-finetuned-cnn_daily_mail-summarization').to(device)

def resumo(documento):
    inputs = tokenizer([documento], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    output = model.generate(input_ids, attention_mask=attention_mask)

    return f" Resumo: {tokenizer.decode(output[0], skip_special_tokens=True)}"

resumo(documento)


tokenizer_config.json:   0%|          | 0.00/348 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.63k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/93.7M [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "google/bert_uncased_L-4_H-256_A-4",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 4,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.47.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

Config of the decoder: <class 'transformers.models.bert.modeling_bert.BertLMHeadModel'> is overwritten by shared decoder config: BertConfig {
  "_name_or_path": "google/bert_uncased_L-4_H-256_A-4",
  "add_cross_attention": true,
  "attention_probs_dropout_prob": 

' Resumo: e um pacote de aplicativos para escritorio e servicos desenvolvido pela microsoft, que contem programas como processador de e - mails, entre outros. anunciado por bill gates embarda, e - versoes direcionadas.'

## Análise de Sentimento

In [5]:

analise_sentimento = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

documento = """
Microsoft  é uma empresa fantástica, que contém produtos maravilhosos que facilitam nosso dia-a-dia.
"""

resultados = analise_sentimento(documento)

def mapear_sentimento(label):
    estrelas = int(label.split()[0])
    if estrelas in [1, 2]:
        return "Negativo"
    elif estrelas == 3:
        return "Neutro"
    else:
        return "Positivo"

# Processa o resultado
for idx, resultado in enumerate(resultados):
    sentimento = mapear_sentimento(resultado['label'])
    print(f"Texto {idx + 1}: {documento}")
    print(f"Classificação: {sentimento} (Confiança: {resultado['score']:.2f})\n")



config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


Texto 1: 
Microsoft  é uma empresa fantástica, que contém produtos maravilhosos que facilitam nosso dia-a-dia.

Classificação: Positivo (Confiança: 0.93)



##### inferência

Modelo Usado:

    O modelo nlptown/bert-base-multilingual-uncased-sentiment é uma variante do BERT treinada para análise de sentimentos em vários idiomas. Ele retorna a probabilidade de diferentes níveis de sentimento (de 1 a 5 estrelas).
        1-2 estrelas: "Negativo".
        3 estrelas: "Neutro".
        4-5 estrelas: "Positivo".